In [1]:
# В РЕЖИМІ РЕАЛЬНОГО ЧАСУ
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import math

def nota(x):
# перелік нот в порядку збільшення її частоти
    noty=['До','До#','Ре','Ре#','Мі','Фа','Фа#','Соль','Соль#','Ля','Ля#','Ci', 'До-2', 'До#-2','Ре-2','Ре#-2','Мі-2','Фа-2','Фа#-2','Соль-2','Соль#-2','Ля-2','Ля#-2','Ci-2', 'До-3']
# коефіцієнт рівномірно-темперованої шкали; у скільки раз ноти відрізняються одна від одної
    p=2**(1/12)

# створюємо масив частот і їх назв (назв нот)
    muz=[]
    for i in range(len(noty)):
        muz.append([440*(p**i)/(p**9), noty[i]])
    
    riz=[]
    for i in range(len(muz)):
        riz.append(abs(muz[i][0]-x))
    if min(riz)>60:
        s='--'
    else:
        s=muz[riz.index(min(riz))][1]
    return s

%matplotlib tk

CHUNK = 1024 * 8           
FORMAT = pyaudio.paInt16    
CHANNELS = 1                
RATE = 44100              

dBV = -40.0                #-47
dBV0 = dBV+(120-94)        #-21
print('dBV',dBV,dBV0)

a=np.power(10.0,dBV/20.0)   #4.47e-3  V/Pa
a0=np.power(10.0,dBV0/20.0) #89.13e-3
print('a',a,a0)

b0=np.power(2.0,15)        #32768
b=b0*a/a0                  #1642.29
print('b',b,b0)

c=a0/b0
print('c',c)

# створюємо фігуру, координатні осі
fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 3))
 
p = pyaudio.PyAudio()
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK)
 
x = np.arange(0, 2 * CHUNK, 2)  #від 0 до 2*CHUNK з кроком 2    
xf = np.linspace(0, RATE, CHUNK)  #від 0 до RATE розмістити CHUNK точок 

print(len(x), len(xf),'------')
# a='5',    a - кортеж з одним елементом

# x, y = [8, 4]    x,y - числа
# z, = [3]     z - число
# або так [z] = [3]     z - число

#тиск від часу 
#заповнюємо випадковими значеннями масиви розміром CHUNK
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)
# plot повертає одноелементный список, який распаковується в line
#або так [line]=...
# об'єкт line створює посилання на дані графіка ax1

#АЧХ, логарифмічна вісь Ох
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

ax1.set_title('Графік зміни тиску від часу')
ax1.set_xlabel('фрагмент')
ax1.set_ylabel('значення')

#обмеження по осі Ox, Оу для першого графіка
ax1.set_ylim(-15000, 15000)
ax1.set_xlim(0, 512)

#підписуємо основні точки осі першого графіка
plt.setp(ax1, xticks=[0, 256, 512], yticks=[-10000, 0, 10000])
# ax1.set_yscale('log')

#обмеження по осі Ox другого графіка від 1 до 20000
ax2.set_xlim(1, RATE / 2)
ax2.set_ylim(0, 7000)
# ax2.set_yscale('log')

print('розпочався потік')

# кількість проаналізованих фрагментів по CHUNK
frame_count = 0
s=0

annotation = ax2.annotate('', xy=(0,0)) 
annotation2 = ax2.annotate('', xy=(0,0)) 

i=0
# while True:
#     i=i+1
for i in range(50):

    stream.start_stream()
    data = np.frombuffer(stream.read(CHUNK),dtype=np.int16)
    stream.stop_stream()
    data_int =data
    data_np = data

    #оновити дані
    line.set_ydata(data_np)
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (0.5 * CHUNK))
    # y1.append(abs(y_1[i])/(L/2))
    # L=CHUNK
#     yf=yf[0:4096]
    yf2=[]
    for k in range(len(yf)):
        yf2.append(abs(yf[k])/(0.5 * CHUNK))
    yf1=[]
    for k in range(1,len(yf2)-1):
        if yf2[k]>yf2[k-1] and yf2[k]>yf2[k+1]:
            yf1.append([yf2[k],xf[k]])
    yf1=sorted(yf1, key=lambda yf1: yf1[0], reverse=True)
    
    #анотація
    #ax2.annotate(str(max1), (max1,max2))
    #round
    new_x=yf1[0][1]
    new_y=yf1[0][0]
    annotation.set_position((new_x,new_y))
    annotation.xy = (new_x,new_y)
    s1='['+str(round(new_x))+'Гц'+'; '+str(round(new_y))+']'+str(nota(new_x))
    annotation.set_text(s1)
    
    new_x=yf1[1][1]
    new_y=yf1[1][0]
    annotation2.set_position((new_x,new_y))
    annotation2.xy = (new_x,new_y)
    s2='['+str(round(new_x))+'Гц'+'; '+str(round(new_y))+']'+str(nota(new_x))
    annotation2.set_text(s2)
    
    print('N=',i)
    print(s1)
    print(s2)
    
 #пробуємо візуалізувати дані
    try:
         # перерисувати фігуру
        #нарисувати полотно
        fig.canvas.draw()
        # очистити всі очікувані події графічного інтерфейсу, перефарбувавши екран, якщо потрібно
        fig.canvas.flush_events()
        
        #збільшити frame_count на 1
        frame_count += 1
   
    # щоб уникнути помилок при відсутності даних
    # tkinter.TclError
    except TclError:
        print('потік зупинився')

        # відсутність операцій pass 
        # frame_rate = frame_count / (time.time() - start_time)
       
        # print('середня частота кадрів = {:.0f} FPS'.format(frame_rate))
        # виходимо з циклу 
        break
        
        # matplotlib.org

dBV -40.0 -14.0
a 0.01 0.19952623149688797
b 1642.2903271498458 32768.0
c 6.0890573576931144e-06
8192 8192 ------
розпочався потік
N= 0
[16410Гц; 0]--
[27695Гц; 0]--
N= 1
[48Гц; 0]--
[44057Гц; 0]--
N= 2
[673Гц; 0]Мі-2
[43432Гц; 0]--
N= 3
[65Гц; 0]--
[44041Гц; 0]--
N= 4
[48Гц; 0]--
[44057Гц; 0]--
N= 5
[4469Гц; 0]--
[39637Гц; 0]--
N= 6
[54Гц; 0]--
[44052Гц; 0]--
N= 7
[92Гц; 0]--
[44014Гц; 0]--
N= 8
[528Гц; 0]До-2
[43578Гц; 0]--
N= 9
[118Гц; 0]--
[43987Гц; 0]--
N= 10
[4216Гц; 0]--
[39890Гц; 0]--
N= 11
[97Гц; 1]--
[44008Гц; 1]--
N= 12
[129Гц; 0]--
[43976Гц; 0]--
N= 13
[92Гц; 0]--
[44014Гц; 0]--
N= 14
[81Гц; 0]--
[44025Гц; 0]--
N= 15
[743Гц; 1]Фа#-2
[43362Гц; 1]--
N= 16
[802Гц; 0]Соль-2
[43303Гц; 0]--
N= 17
[888Гц; 0]Ля-2
[43217Гц; 0]--
N= 18
[894Гц; 0]Ля-2
[43212Гц; 0]--
N= 19
[4286Гц; 0]--
[39820Гц; 0]--
N= 20
[1357Гц; 0]--
[42749Гц; 0]--
N= 21
[102Гц; 0]--
[44003Гц; 0]--
N= 22
[1055Гц; 0]До-3
[43050Гц; 0]--
N= 23
[113Гц; 0]--
[43992Гц; 0]--
N= 24
[1044Гц; 224]До-3
[43061Гц; 224]--
N= 25
